In [1]:
from wagtailkit.academic.models import ResourceManagementUnit, Course
from wagtailkit.teachers.models import Teacher

rmu = ResourceManagementUnit.objects
fik = rmu.get(code='FIK')
prodi_fik = fik.get_children()

for prodi in prodi_fik:
    print(prodi.name)
    for mk in prodi.courses.all():
        print('--' + mk.name)

Teknik Informatika
--Pengantar Sistem Dijital
--Pengantar Organisasi Komputer
--Matematika Dasar 2
--Pemrograman Lanjut
--Teori Bahasa & Automata
--Rekayasa Perangkat Lunak
--Pemrograman Sistem
--Sistem Cerdas
--Jaringan Komputer
--Proyek Perangkat Lunak
--Data Science & Analytics
--Analisis Numerik
--Desain & Analisis Algoritma
--Basis Data Lanjut
--Teknik Kompilator
--Grafika Komputer
--Kriptografi & Keamanan Informasi
--Layanan & Aplikasi Web
--Pemrograman Paralel
--Teknologi Mobile
--Penjaminan Mutu Perangkat Lunak
--Penambangan Data
Sistem Informasi
--Administrasi Bisnis
--Prinsip - Prinsip Manajemen
--Sistem - Sistem Perusahaan
--Sistem Informasi Akuntansi dan Keuangan
--Sistem Interaksi
--Analisis dan Perancangan Sistem Informasi
--Arsitektur & Pemrograman Aplikasi Perusahaan
--Manajemen Proyek TI
--Jaringan Komunikasi Data
--Proyek Pengembangan Sistem Informasi
--Komunikasi Bisnis dan Teknis
--Statistika Terapan
--Manajemen Sistem Informasi
--Manajemen Layanan TI
--Administrasi

In [2]:
strip = '-'

def get_child(instance):
    for child in instance.get_children():
        print(strip * child.level + child.name + ': ' + str(child.courses.count()))
        for mk in child.courses.all():
            print('------' + mk.name)
        get_child(child)
        
for mu in rmu.filter(level=0):
    print(strip * mu.level + mu.name)
    get_child(mu)

IBI Darmajaya
-Fakultas Ilmu Komputer: 18
------Dasar - Dasar Arsitektur Komputer
------Prinsip - Prinsip Sistem Informasi
------Matematika Dasar
------Fisika Dasar
------Matematika Diskret 1
------Dasar-Dasar Pemrograman 1
------Dasar-Dasar Pemrograman 2
------Matematika Diskret 2
------Aljabar Linier
------Statistika dan Probabilitas
------Perancangan & Pemrograman Web
------Struktur Data & Algoritma
------Sistem Operasi
------Basis Data
------Kerja Praktik
------Manusia dan Komputer
------Tugas Akhir
------Metodologi Penelitian & Penulisan Ilmiah
--Teknik Informatika: 22
------Pengantar Sistem Dijital
------Pengantar Organisasi Komputer
------Matematika Dasar 2
------Pemrograman Lanjut
------Teori Bahasa & Automata
------Rekayasa Perangkat Lunak
------Pemrograman Sistem
------Sistem Cerdas
------Jaringan Komputer
------Proyek Perangkat Lunak
------Data Science & Analytics
------Analisis Numerik
------Desain & Analisis Algoritma
------Basis Data Lanjut
------Teknik Kompilator
------G

In [3]:
sip = rmu.get(code='SIP')
ancestors = sip.get_ancestors(ascending=False, include_self=True)
for ancestor in ancestors:
    sep = '' if ancestor is ancestors[0] else ' > '
    print(sep + ancestor.name)

IBI Darmajaya
 > Fakultas Ilmu Komputer
 > Sistem Informasi


In [4]:
strip = '-'

def get_child(instance):
    for child in instance.get_children():
        print(strip * child.level + child.name + ': ' + str(child.teacher_count))
        
        get_child(child)

rmu_with_teacher_count = rmu.add_related_count(rmu.all(), Teacher, 'homebase', 'teacher_count', cumulative=True)
        
for mu in rmu_with_teacher_count:
    print(strip * mu.level + mu.name + ' - Teacher : {}'.format(mu.teacher_count))
    for teacher in mu.teachers.all():
        print('------' + teacher.person.fullname + ', NIDN: ' + teacher.tid)

FieldDoesNotExist: Teacher has no field named 'homebase'

In [ ]:
from django.db import models
from wagtailkit.academic.models import ResourceManagementUnit, Course
from wagtailkit.teachers.models import Teacher

strip='-'
rmu = ResourceManagementUnit.objects

rmus = rmu.annotate(
    active_courses=
            models.Subquery(
                Course.objects.filter(
                    models.Q(rmu__tree_id=models.OuterRef('tree_id')) 
                    & models.Q(rmu__lft__gte=models.OuterRef('lft')) 
                    & models.Q(rmu__lft__lte=models.OuterRef('rght'))
                ).order_by().values('rmu__tree_id').annotate(
                    total=models.Count('*')
                ).values('total'),
                output_field=models.IntegerField()
           )
)

for rmu in rmus:
    print(rmu.name + ' course:' + str(rmu.active_courses))

In [ ]:
for rm in ResourceManagementUnit.objects.get_summary():
    print("{}{} {}:{} total:{}".format(strip * rm.level, str(rm.name), str(rm.code), str(rm.total_teachers), str(rm.total_cumulative_courses)))